In [ ]:
#--- Import Files ---

import tensorflow as tf
from tensorflow.kera.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc

In [ ]:
# --- Paths ---
train_dir = 'water_dataset/train'
test_dir = 'water_dataset/test'

In [ ]:
# --- Data Augmentation ---
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    shear_range=0.15
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=8,
    class_mode='binary'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=8,
    class_mode='binary',
    shuffle=False
)

In [ ]:

# --- Transfer Learning Model ---

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze pretrained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:

# --- Callbacks ---
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

# --- Train ---
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=20,
    callbacks=[es, rlr]
)
# Evaluate the model
loss, acc = model.evaluate(test_gen)
print(f"Test Accuracy: {acc*100:.2f}%")

In [ ]:
# PLot Accuracy & Loss

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history_ft.history['accuracy'], label='Train')
plt.plot(history_ft.history['val_accuracy'], label='Validation')
plt.title("Model Accuracy"); plt.legend()
plt.subplot(1,2,2)
plt.plot(history_ft.history['loss'], label='Train')
plt.plot(history_ft.history['val_loss'], label='Validation')
plt.title("Model Loss"); plt.legend()
plt.show()


In [ ]:
# Confusion Matrix

Y_pred = model.predict(test_gen)
y_pred = (Y_pred > 0.5).astype(int)

cm = confusion_matrix(test_gen.classes, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted"); plt.ylabel("Actual")
plt.show()

In [ ]:
# ROC Curve

fpr, tpr, thresholds = roc_curve(test_gen.classes, Y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
plt.plot([0,1], [0,1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.show()

print(f"ROC-AUC Score: {roc_auc:.3f}")


In [ ]:
print("\nClassification Report:\n")
print(classification_report(test_gen.classes, y_pred, target_names=['Clean','Polluted']))

model.save('water_pollution_detector_v2.h5')
print("Fine-tuned model saved as water_pollution_detector_v2.h5")